# Problem 2: $y = 3x^2y + 5xy - x -2y + 3$

In [2]:
import numpy as np
import random


L = np.array([[ 0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[ 0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[  0,0,0,0,1, 0],
               [ 0,0,0,0,0, 1],
               [-3,1,1,2,0,-1]])

x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

211325421 422 394 534252 210495288


In [13]:
import galois
import numpy as np
import random
GF = galois.GF(113)

L_ = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R_ = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O_ = np.array([[    0,0,0,0,1, 0],
               [    0,0,0,0,0, 1],
               [113-3,1,1,2,0, 113-1]])

x = GF(random.randint(1,112))
y = GF(random.randint(1,112))

# out = 3 * x * x * y + 5 * x * y - x- 2 * y + 3# the witness vector with the intermediate variables inside
v1 = x * x
v2 = v1 * y
out = GF(3) * v2 + GF(5) * x * y - x - GF(2) * y + GF(3) # out = 3 * x * x * y + 5 * x * y - x- 2 * y + 3
out = GF(out)
print(out)
witness = GF(np.array([1, out, x, y, v1, v2]))

Lgf = GF(L_)
Rgf = GF(R_)
Ogf = GF(O_)
print(Lgf)

105
[[0 0 3 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 0 0 0]]


In [12]:
# interpolate each matrix column at x=1, x=2, x=3. This gives us the polynomials
# galois.lagrange_poly takes 2 GFarray input. [1,2,3] and [f(1),f(2),f(3)]
target = GF(np.array([1,2,3]))
def interpolate_columns(column):
    return galois.lagrange_poly(target, column)

Lpoly = np.apply_along_axis(interpolate_columns, 0, Lgf)
Rpoly = np.apply_along_axis(interpolate_columns, 0, Rgf)
Opoly = np.apply_along_axis(interpolate_columns, 0, Ogf)

print(Lpoly)


[Poly(0, GF(113)) Poly(0, GF(113)) Poly(2x^2 + 104x + 10, GF(113))
 Poly(0, GF(113)) Poly(112x^2 + 4x + 110, GF(113)) Poly(0, GF(113))]


### We now interpolate each column of the L matrix by selecting x values 1,2,3 arbitrarily
```
L Matrix
[0 0 3 0 0 0]
[0 0 0 0 1 0]
[0 0 1 0 0 0]
```

### Create equation for points on the curve
```
[0,0,0] = (1,0),(2,0),(3,0) = 0
[0,0,0] = (1,0),(2,0),(3,0) = 0
[3,0,1] = (1,3),(2,0),(3,1)
[0,0,0] = (1,0),(2,0),(3,0) = 0
[0,1,0] = (1,0),(2,1),(3,0)
[0,0,0] = (1,0),(2,0),(3,0) = 0
```

For 
[3,0,1] = (1,3),(2,0),(3,1)  
3 points = degree 2 quadratic i.e. $y = ax^2 + bx + c$ that passes through these points  
First point (1,3):   $f(1) = a(1)^2 + b(1) + c = 3$  
Second point (2,0):  $f(2) = a(1)^2 + b(1) + c = 0$  
Third point (3,1):   $f(3) = a(1)^2 + b(1) + c = 1$  

Line equation is $2x^2 -9x + 10$ which in Galois Field is $2x^2 +(104)x + 10$  

Repeat for all columns. Here is the result  
```
Poly(0, GF(113)) 
Poly(0, GF(113)) 
Poly(2x^2 + 104x + 10, GF(113))
Poly(0, GF(113)) 
Poly(112x^2 + 4x + 110, GF(113)) 
Poly(0, GF(113))
```

We can now Hadamard Product the polynomials with the witness vector [1, out, x, y, v1, v2]  
```
1 * Poly(0, GF(113)) +                
out * Poly(0, GF(113)) +                
x * Poly(2x^2 + 104x + 10, GF(113)) +
y * Poly(0, GF(113)) + 
v1 * Poly(112x^2 + 4x + 110, GF(113)) + 
v2 * Poly(0, GF(113))
```
i.e. $xValue * (2x^2 + 104x + 10)$ + $v1Value * (112x^2 + 4x + 110)$  

Finally we have 1 polynomial representing the entire L matrix and witness

In [14]:
from sympy import symbols, Eq, solve

# Define the symbols
a, b, c = symbols('a b c')

# Define the equations based on the points
eq1 = Eq(a * 1**2 + b * 1 + c, 3)
eq2 = Eq(a * 2**2 + b * 2 + c, 0)
eq3 = Eq(a * 3**2 + b * 3 + c, 1)

# Solve the system of equations
solution = solve((eq1, eq2, eq3), (a, b, c))
solution # {a: 2, b: -9, c: 10}

{a: 2, b: -9, c: 10}

In [11]:
from functools import reduce
def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(Lpoly, witness)
term_2 = inner_product_polynomials_with_witness(Rpoly, witness)
term_3 = inner_product_polynomials_with_witness(Opoly, witness)

print(term_1)
print(term_2)
print(term_3)

51x^2 + 38x + 89
68x^2 + 51x + 91
60x^2 + 67x + 16


In [24]:
# t = (x-1)(x-2)(x-3)
# h = (term_1 * term_2 - term_3) // t
t = galois.Poly([1,112],field=GF) * galois.Poly([1,111],field=GF) * galois.Poly([1,110],field=GF)

h = (term_1 * term_2 - term_3) // t

print(t)
print(h)
print(h *t) # 78x^4 + 100x^3 + 49x^2 + 17x + 95
print(term_1 * term_2)
print(term_3 + h * t)

# assert term_1 * term_2 == term_3 + h * t, "division has a remainder"

x^3 + 107x^2 + 11x + 107
78x + 3
78x^4 + 100x^3 + 49x^2 + 17x + 95
78x^4 + 100x^3 + 88x^2 + 87x + 76
78x^4 + 100x^3 + 109x^2 + 84x + 111
